In [1]:
import numpy as np
import pandas as pd
import scipy.special as sp
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import PIL

rng = np.random.default_rng(42)

# Get and visualise data

In [3]:
# df = pd.read_csv('fer2013.csv')
import PIL.Image


print(df.head())
print(f'number of photos in the dataset is: {len(df)}')
      
X = []
Y = []

for row in df.index:  # correct this!!!
    X.append(list(map(int, df.iloc[row].pixels.split(' '))))
    Y.append(df.iloc[row].emotion)

X = np.array(X) / 255 # normalise pixel values to lie between 0 and 1
Y = np.array(Y)

print(f'check number of features is 48**2: {X.shape[1] == 48**2}')

X, Y = shuffle(X, Y, random_state=42)  # numpys shuffle is not nice because you can't shuffle two arrays simultaneously

train_proportion = 0.7
train_index = int(0.7*len(X))

X_train, X_test = X[:train_index], X[train_index:]
Y_train, Y_test = Y[:train_index], Y[train_index:]
# or just use train_test_split from sklearn.model_selection for the same effect

print(f'Number of samples in training set: {len(X_train)}')
print(f'Number of samples in test set: {len(X_test)}')

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
number of photos in the dataset is: 35887
check number of features is 48**2: True
Number of samples in training set: 25120
Number of samples in test set: 10767


In [4]:
emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def show_sample_image(emotion: int):
    emotion_no = emotions.index(emotion)
    df_emotion = df[df.emotion == emotion_no]

    random_no = rng.integers(0, len(df_emotion))
    print(random_no)
    print(df_emotion.iloc[random_no])

    img = np.array(list(map(int, df_emotion.iloc[random_no].pixels.split(' '))), dtype=np.uint8).reshape((48,48))
    img = PIL.Image.fromarray(img).resize((1000, 1000))
    img.show()

show_sample_image('Surprise')

357
emotion                                                    5
pixels     243 255 115 9 21 19 23 24 27 29 31 35 37 39 49...
Usage                                               Training
Name: 3285, dtype: object


# Train and predict

In [5]:
# X - input matrix, size NxD
# Z - hidden layer, size NxM
# Y - output layer, size NxK
# T - target one-hot encoded as an indicator matrix, size NxK
# W2 - weight matrix of the second layer
def derivative_W1(X, Z, W2, Y, T):
    return X.T.dot((T - Y).dot(W2.T) * Z * (1 - Z))
def derivative_b1(Z, W2, Y, T):
    return np.sum((T - Y).dot(W2.T) * Z * (1 - Z), axis=0)
def derivative_W2(Z, Y, T):
    return Z.T.dot(T - Y)
def derivative_b2(Y, T):
    return np.sum(T - Y, axis=0)

def feedforward(X, W1, b1, W2, b2):
    a = X.dot(W1) + b1
    Z = np.tanh(a)
    alpha = Z.dot(W2) + b2
    Y = sp.softmax(alpha, axis=1)
    return Z, Y

def predict(pY):
    return np.argmax(pY, axis=1)

In [8]:
class ANN(object):
    def __init__(self, M) -> None:
        self.M = M

    def fit(self, X_train, Y_train, X_test, Y_test, learning_rate, penalty, steps):
        K = len(emotions)
        N, D = X_train.shape

        # create the indicator matrix for the targets
        T = np.zeros((N, K), dtype=int)
        T_test = np.zeros((len(Y_test), K), dtype=int)
        for ii in range(N):
            T[ii, Y_train[ii]] = 1
        for ii in range(len(Y_test)):
            T_test[ii, Y_test[ii]] = 1

        # randomly initialize weights
        self.W1 = np.random.randn(D, self.M) / np.sqrt(D)
        self.b1 = np.random.randn(self.M)
        self.W2 = np.random.randn(self.M, K) / np.sqrt(self.M)
        self.b2 = np.random.randn(K)

        cost = []

        for ii in range(steps):
            Z, pY_train = feedforward(X_train, self.W1, self.b1, self.W2, self.b2)
            print(pY_train)
            print(predict(pY_train))
            break

            self.W1 -= learning_rate * (derivative_W1(X_train, Z, self.W2, pY_train, T) + penalty*self.W1)
            self.b1 -= learning_rate * (derivative_b1(Z, self.W2, pY_train, T) + penalty*self.b1)
            self.W2 -= learning_rate * (derivative_W2(Z, pY_train, T) + penalty*self.W2)
            self.b2 -= learning_rate * (derivative_b2(pY_train, T) + penalty*self.b2)

            if ii % 10 == 0:
                _, pY_test = feedforward(X_test, self.W1, self.b1, self.W2, self.b2)
                preds = predict(pY_test)
                print(preds)
                print(Y_test)
                classification_rate = np.mean(preds == Y_test)
                print(f'classification rate: {classification_rate}')
                cost.append(np.sum(T_test * np.log(pY_test)))

        # plt.plot(cost)
        # plt.show()



In [9]:
model = ANN(100)
model.fit(X_train, Y_train, X_test, Y_test, learning_rate=1e-6, penalty=0, steps=1000)

[[0.05438607 0.17616684 0.1257669  ... 0.38452182 0.02207647 0.20858666]
 [0.05184157 0.19787179 0.13492912 ... 0.341275   0.02596001 0.22186059]
 [0.05626767 0.16232185 0.15868729 ... 0.32856727 0.0244281  0.24207795]
 ...
 [0.04740451 0.13304035 0.11882646 ... 0.42874405 0.02068709 0.22404237]
 [0.05738611 0.16175321 0.15369372 ... 0.3780483  0.02029    0.19960927]
 [0.04401344 0.12276329 0.10811382 ... 0.40499571 0.0167143  0.28008531]]
[4 4 4 ... 4 4 4]
